## Set-up

In [1]:
import tifffile
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from pathlib import Path
import gc
from matplotlib import colormaps as cm
from glob import glob

import sys
import os

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

import utils
import track

2024-05-08 13:42:14.972766: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-08 13:42:15.000240: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 13:42:15.000270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 13:42:15.001170: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 13:42:15.006083: I tensorflow/core/platform/cpu_feature_guar

### Hyper parameters

In [2]:
loadstar_weight_name = "loadstar_first"
magik_weight_name = "MAGIK_MP_MPN"

# Experiment parameters
dir_mode = True
input_path = r'/home/gideon/data/data_VE/test' # Change this to where you have the tiff file
output_path = r'/home/gideon/Documents/PhD/Projects/KAW/output' # Change this to where you want the output to be saved
pages_batch_size = 2
channel_width = 0.3 # Microns

# Loadstar parameters
alpha = 0.999
cutoff = 0.00000002

# MAGIK parameters
radius = 0.08 # Value between 0 and 1 where 1 is the full image hight
traj_min = 6
n_frames = 4

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

loadstar, magik = utils.load_models(loadstar_weight_name, magik_weight_name)

if dir_mode:
    image_paths = glob(input_path + '/*.tif')
else:
    image_paths = [input_path]

Num GPUs Available:  1


2024-05-08 13:42:17.568588: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-08 13:42:17.568745: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-08 13:42:17.568817: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
for path in image_paths:
    print(path)

/home/gideon/data/data_VE/test/Experiment-16031.tif
/home/gideon/data/data_VE/test/Experiment-22070.tif


#### Test loadstar parameters

In [9]:
utils.test_loadstar(image_paths[0], loadstar, alpha, cutoff, 1)

2024-05-06 11:50:51.012675: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


[]

5dc3a68c-e34e-4080-9c3e-2a532b2ccb4d6.29.35dc3a68c-e34e-4080-9c3e-2a532b2ccb4d


### Extract metadata

In [4]:
tiffpage = tifffile.TiffFile(image_paths[0]).pages[0]
for tag in tiffpage.tags.values():
    print(tag.name, tag.value)

NewSubfileType 0
ImageWidth 1254
ImageLength 1254
BitsPerSample 16
Compression 1
PhotometricInterpretation 1
ImageDescription ImageJ=1.54f
images=220
frames=220
unit=micron
finterval=0.45014611872146115
loop=false
min=254.0
max=255.0
StripOffsets (19051,)
SamplesPerPixel 1
RowsPerStrip 1254
StripByteCounts (3145032,)
XResolution (9090909, 1000000)
YResolution (9090909, 1000000)
ResolutionUnit 1
IJMetadataByteCounts (12, 78, 78, 78, 78, 78, 78, 78, 78, 78, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 

In [5]:
width = tiffpage.tags.get('ImageWidth').value # Pixels
height = tiffpage.tags.get('ImageLength').value # Pixels
x_resolution = tiffpage.tags.get('XResolution').value # Pixels per micron
y_resolution = tiffpage.tags.get('YResolution').value # Pixels per micron

### Calculated paramters

In [6]:
# Calculated parameters
x_resolution = x_resolution[0] / 1000000 # Pixels per micron
y_resolution = y_resolution[0] / 1000000 # Pixels per micron

pixel_size_x = 1/x_resolution # Microns per pixel
pixel_size_y = 1/y_resolution # Microns per pixel

channel_width_pixels = channel_width * x_resolution # Pixels
channel_width_norm = channel_width_pixels / width

radius_ratio = radius / channel_width_norm

print(f"Channel width: {channel_width_pixels} pixels")
print(f"Radius ratio: {radius_ratio} ratio")

Channel width: 2.7272727 pixels
Radius ratio: 36.78400036784 ratio


## Detect

In [7]:
import importlib
importlib.reload(utils)
detections  = utils.detect(image_paths, loadstar, pages_batch_size, alpha, cutoff)
utils.save_detections(detections, f"{output_path}/detections.csv", full=True)

  0%|          | 0/250 [00:00<?, ?it/s]

2436


100%|██████████| 250/250 [02:18<00:00,  1.80it/s]


2133
Predicting trajectories...
Creating graph edges...
Creating dataframe...
Getting trajectories...
Done!
Predicting trajectories...
Creating graph edges...
Creating dataframe...
Getting trajectories...
Done!


## Track

In [8]:
import importlib
importlib.reload(track)

detections = utils.load_detections(f"{output_path}/detections.csv")

detections = utils.warp_axis(detections, radius_ratio, axis_name='centroid-0')
tracks = track.predict_trajectories(magik, detections, radius, n_frames, traj_min)
tracks = utils.unwarp_axis(tracks, radius_ratio, axis_name='x')

# Convert to pixel values
tracks['x'] *= width
tracks['y'] *= height
track.save_trajectories(tracks, f"{output_path}/tracks.csv")

100%|██████████| 1/1 [00:03<00:00,  3.69s/it]
/home/gideon/Documents/PhD/Projects/KAW/.venv/lib/python3.11/site-packages/deeptrack/models/gnns/utils.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  d.loc[
/home/gideon/Documents/PhD/Projects/KAW/.venv/lib/python3.11/site-packages/deeptrack/models/gnns/utils.py:52: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  alpha = 0.1 + d_alpha * int(solutions[i : i + 1]["frame_y"])
/home/gideon/Documents/PhD/Projects/KAW/.venv/lib/python3.11/site-packages/deeptrack/models/gnns/utils.py:54: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if not (str(int(solutions[i : i + 1]["node_x"])

### Plot track

In [9]:
tracks.head()

set  frame           y           x  entity
3    0    0.0  251.217419  436.453195     1.0
4    0    0.0  257.965005  213.490979     2.0
6    0    0.0  306.162532  437.811566     7.0
7    0    0.0  515.116151  606.502028     3.0
11   0    0.0  757.485722  458.801431     4.0

In [10]:
import importlib
importlib.reload(track)
track.plot_trajectories(tracks, image_paths, output_path, pages_batch_size)

 32%|███▏      | 81/250 [01:53<03:56,  1.40s/it]Exception ignored in: <function TransformNode.set_children.<locals>.<lambda> at 0x7d12b01309a0>
Traceback (most recent call last):
  File "/home/gideon/Documents/PhD/Projects/KAW/.venv/lib/python3.11/site-packages/matplotlib/transforms.py", line 198, in <lambda>
    self, lambda _, pop=child._parents.pop, k=id_self: pop(k))

KeyboardInterrupt: 
100%|██████████| 250/250 [05:51<00:00,  1.41s/it]


## Further analyze the tracks

### Change x and y units

#### Add axis for lengths in microns

In [51]:
tracks = track.load_trajectories(f"{output_path}/tracks.csv")

tracks['y_microns'] = tracks['y'] * pixel_size_y
tracks['x_microns'] = tracks['x'] * pixel_size_x

#### Alt. replace x and y with micron versions

In [ ]:
tracks = track.load_trajectories(f"{output_path}/tracks.csv")

tracks['y'] = tracks['y'] * pixel_size_y
tracks['x'] = tracks['x'] * pixel_size_x

In [ ]:
tracks.head(20)

### Remove still particles

In [52]:
still_cuttoff = 40

print(f'Number of objects before: {len(tracks[tracks["set"] == 1].entity.unique())}')

tracks_stills_removed = track.remove_still_objects(tracks, still_cuttoff)

print(f'Number of objects after: {len(tracks_stills_removed[tracks_stills_removed["set"] == 1].entity.unique())}')

Number of objects before: 88
Number of objects after: 76


### Combine tracks

In [37]:
track.plot_trajectories(tracks_stills_removed, image_paths, output_path, pages_batch_size, stop=50)

100%|██████████| 26/26 [00:35<00:00,  1.37s/it]


### Track length and distance

In [53]:
tracks.head(20)

set  frame            y            x  entity   y_microns   x_microns
0     0    0.0   251.217419   436.453195     1.0   27.633916   48.009852
1     0    0.0   257.965005   213.490979     2.0   28.376151   23.484008
2     0    0.0   306.162532   437.811566     7.0   33.677879   48.159273
3     0    0.0   515.116151   606.502028     3.0   56.662777   66.715224
4     0    0.0   757.485722   458.801431     4.0   83.323430   50.468158
5     0    0.0  1025.031262    31.761879     5.0  112.753440    3.493807
6     0    0.0  1072.599746  1038.329180     6.0  117.985973  114.216211
7     0    1.0   251.193142   435.768285     1.0   27.631246   47.934512
8     0    1.0   273.037082   213.066413     2.0   30.034079   23.437306
9     0    1.0   549.093269   606.675958     3.0   60.400260   66.734356
10    0    1.0   768.506125   459.504173     4.0   84.535675   50.545460
11    0    1.0  1020.459730    32.261081     5.0  112.250571    3.548719
12    0    1.0  1082.629739  1038.129292     6.0  119.089273  114.194223
13    0    2.0   251.576414   436.261333     1.0   27.673406   47.988747
14    0    2.0   297.490082   213.938382     2.0   32.723909   23.533222
15    0    2.0   304.932547   438.344302     7.0   33.542580   48.217874
16    0    2.0   551.067165   606.231164     3.0   60.617389   66.685429
17    0    2.0   798.953421   458.535157     4.0   87.884877   50.438868
18    0    2.0  1020.950997    31.544760     5.0  112.304611    3.469924
19    0    2.0  1095.599736  1038.276010     6.0  120.515972  114.210362

In [54]:
tracks = tracks_stills_removed
tracks = track.distance_traveled(tracks)

tracks = tracks.sort_values(by=['set', 'entity', 'frame'], ascending=True).head(20)
tracks.head(20)

set  frame           y           x  entity  y_microns  x_microns  \
1      0    0.0  257.965005  213.490979     2.0  28.376151  23.484008   
8      0    1.0  273.037082  213.066413     2.0  30.034079  23.437306   
14     0    2.0  297.490082  213.938382     2.0  32.723909  23.533222   
22     0    3.0  309.217778  213.569442     2.0  34.013956  23.492639   
29     0    4.0  314.099161  213.122354     2.0  34.550908  23.443459   
39     0    5.0  334.301202  213.760263     2.0  36.773133  23.513629   
48     0    6.0  363.133069  214.306393     2.0  39.944638  23.573703   
58     0    7.0  377.520901  214.377858     2.0  41.527300  23.581565   
69     0    8.0  400.820986  214.069462     2.0  44.090309  23.547641   
80     0    9.0  412.917597  214.229096     2.0  45.420936  23.565201   
92     0   10.0  412.551755  214.240320     2.0  45.380693  23.566435   
105    0   11.0  444.348429  214.713253     2.0  48.878328  23.618458   
120    0   12.0  449.156767  214.413848     2.0  49.407245  23.585524   
133    0   13.0  456.120655  213.572464     2.0  50.173273  23.492971   
157    0   15.0  456.790066  213.360150     2.0  50.246908  23.469617   
3      0    0.0  515.116151  606.502028     3.0  56.662777  66.715224   
9      0    1.0  549.093269  606.675958     3.0  60.400260  66.734356   
16     0    2.0  551.067165  606.231164     3.0  60.617389  66.685429   
23     0    3.0  580.574086  605.721325     3.0  63.863150  66.629346   
30     0    4.0  624.013185  606.494141     3.0  68.641451  66.714356   

       delta_y   delta_x  traveled_y  traveled_x  
1     0.000000  0.000000    0.000000    0.000000  
8    15.072077  0.424567   15.072077    0.424567  
14   24.453000  0.871969   39.525077    1.296536  
22   11.727696  0.368939   51.252773    1.665475  
29    4.881383  0.447089   56.134156    2.112564  
39   20.202040  0.637910   76.336197    2.750473  
48   28.831868  0.546130  105.168064    3.296603  
58   14.387832  0.071465  119.555896    3.368068  
69   23.300085  0.308396  142.855981    3.676465  
80   12.096611  0.159634  154.952592    3.836099  
92    0.365842  0.011223  155.318434    3.847322  
105  31.796675  0.472934  187.115108    4.320256  
120   4.808338  0.299405  191.923446    4.619661  
133   6.963888  0.841384  198.887334    5.461045  
157   0.669410  0.212315  199.556745    5.673359  
3     0.000000  0.000000    0.000000    0.000000  
9    33.977117  0.173930   33.977117    0.173930  
16    1.973896  0.444794   35.951014    0.618724  
23   29.506921  0.509839   65.457935    1.128562  
30   43.439099  0.772815  108.897034    1.901378

In [ ]:
import importlib
importlib.reload(track)

tracks = track.count_appearances(tracks)
track.save_trajectories(tracks, f"{output_path}/tracks.csv")

tracks.head(50)

### Extract intensity

In [ ]:
tracks = utils.detection_intensity(image_paths, tracks, pages_batch_size, mode='mean', kernel_size=3)
track_intensities = utils.track_intensity(tracks, mode='mean')

In [ ]:
tracks.head(10)

In [ ]:
print(track_intensities)